Add noise (not done)

In [ ]:
bands_snr = [128, 201, 243, 228, 74, 275, 110, 880, 838, 802, 754, 750, 910, 1087, 586, 516, 167, 57, 250,
        None, None, None, None, None, None, 150, None, None, None, None, None, None, None, None, None, None]

bands_netd = [None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None,
         None, None, 0.05, 0.2, 0.07, 0.07, 0.25, 0.25, None, 0.25, 0.25, 0.05, 0.25, 0.05, 0.05, 0.25, 0.25, 0.25, 0.35]


snr_netd = zip(bands_snr, bands_netd)
all_bands = data_ir_vis[:,1:30]
print(all_bands[0,:])

print(all_bands.shape)

for band_i, (snr, netd) in enumerate(snr_netd):
    if snr != None:
        avg_squared_signal_pwr = np.mean(np.square(all_bands[:, band_i]))
        noise_std = np.sqrt(avg_squared_signal_pwr / snr)
    else:
        noise_std = netd

    all_bands[:, band_i] += np.random.normal(0, noise_std, all_bands[:, band_i].shape)

In [ ]:
import keras.backend as K

def custom_loss(y_true, y_pred):
    # Define the binary cross-entropy loss
    bce_loss = K.binary_crossentropy(y_true, y_pred)
    
    # Define a penalty for false negatives
    # Adjust the value of alpha based on the importance of reducing false negatives
    alpha = 0.5  # Example value, you can adjust as needed
    false_negative_penalty = alpha * K.sum(y_true * K.cast(K.greater(y_pred, 0.5), 'float32'), axis=-1)
    
    # Add the penalty to the loss
    total_loss = bce_loss + false_negative_penalty
    
    return total_loss

In [ ]:
if USE_FIFTY_FIFTY:

    #data_ir_vis, data_test = train_test_split(data_ir_vis, test_size=0.05, random_state=1)

    low_vis = data_ir_vis[np.where(data_ir_vis[:,30] < low_vis_threshold)[0]]

    #raw_data_ir = pd.read_csv('fog_dataset/' + 'fog_data_ir_clear.dat').to_numpy()
    #raw_data_vis = pd.read_csv('fog_dataset/' + 'fog_data_vis_clear.dat').to_numpy()
    #data_ir = np.array([row[0].split() for row in raw_data_ir[25:]])

    
    
    #data_vis = np.array([row[0].split()[1:] for row in raw_data_vis[41:]])

    #tmp_data = np.hstack([data_ir, data_vis])
    #tmp_data = tmp_data[:,:-1] 
    #tmp_data = tmp_data.astype(np.float32)

    high_vis_i = np.where(data_ir_vis[:,30] >= low_vis_threshold)[0]      #tmp_data[:,30]
    if len(low_vis) > len(high_vis_i):
       print('Not enough high vis data for 50/50')
    r_high_vis_i = np.random.choice(high_vis_i, len(low_vis), replace=False)
    fifty_fifty = np.vstack([low_vis, data_ir_vis[r_high_vis_i]])       #tmp_data[r_high_vis_i]]
    data_ir_vis = fifty_fifty

In [1]:
y_train_label = np.zeros_like(y_train, dtype=int)
y_val_label = np.zeros_like(y_val, dtype=int)
y_test_label = np.zeros_like(y_test, dtype=int)

for i in np.where(target_scaler.inverse_transform(y_train) < low_vis_threshold)[0]:
   y_train_label[i] = 1

for i in np.where(target_scaler.inverse_transform(y_val) < low_vis_threshold)[0]:
   y_val_label[i] = 1

for i in np.where(target_scaler.inverse_transform(y_test) < low_vis_threshold)[0]:
   y_test_label[i] = 1

NameError: name 'np' is not defined

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error



USE_MIXED = False
USE_PRESSURE_TEMP = False
noise_std_dev = 1e-3


if USE_MIXED:
    filenames_vis = ['fog_data_vis_clear.dat', 'fog_data_vis_ice.dat', 'fog_data_vis_mixed.dat', 'fog_data_vis_water.dat']
    filenames_ir = ['fog_data_ir_clear.dat', 'fog_data_ir_ice.dat', 'fog_data_ir_mixed.dat', 'fog_data_ir_water.dat']
else:
    filenames_vis = ['fog_data_vis_clear.dat', 'fog_data_vis_ice.dat', 'fog_data_vis_water.dat']
    filenames_ir = ['fog_data_ir_clear.dat', 'fog_data_ir_ice.dat', 'fog_data_ir_water.dat']

target_i = 30 # Visibility
if USE_PRESSURE_TEMP:
    features_i = np.concatenate((np.arange(4, 17), np.arange(20, 30), np.arange(31,36))) # 13 ir bands, 10 vis bands, 3 angles, pressure, temp
else:
    features_i = np.concatenate((np.arange(4, 17), np.arange(20, 30), np.arange(31,34)))



data_ir_vis = []

for file_ir, file_vis in zip(filenames_ir, filenames_vis):
  raw_data_ir = pd.read_csv('fog_dataset/' + file_ir).to_numpy()
  raw_data_vis = pd.read_csv('fog_dataset/' + file_vis).to_numpy()

  data_ir = np.array([row[0].split() for row in raw_data_ir[25:]])
  data_vis = np.array([row[0].split()[1:] for row in raw_data_vis[41:]])

  if len(data_ir_vis) == 0:
    data_ir_vis = np.hstack([data_ir, data_vis])
  else:
    data_ir_vis = np.vstack([data_ir_vis, np.hstack([data_ir, data_vis])])


# Remove surface description to convert to float
data_ir_vis = data_ir_vis[:,:-1] 
data_ir_vis = data_ir_vis.astype(np.double)

nan_i = np.where(np.isnan(data_ir_vis))[0]
data_ir_vis = np.delete(data_ir_vis, nan_i, axis=0)



X = np.zeros((data_ir_vis.shape[0], len(features_i)))
y = np.zeros(data_ir_vis.shape[0])

for i in range(len(data_ir_vis)):
  X[i] = data_ir_vis[i, features_i]
  y[i] = data_ir_vis[i,target_i]

# 19 Data points contains nan. Not sure why. Band 32 culprit, 8th feature
nan_i = np.where(np.isnan(X))[0]
print(f'nani: {nan_i}')
X = np.delete(X, nan_i, axis=0)
y = np.delete(y, nan_i, axis=0)


# Scaling features
feature_scaler = StandardScaler()
X_scaled = feature_scaler.fit_transform(X)

X_scaled_noisy = X_scaled + np.random.normal(0, noise_std_dev, X_scaled.shape)

# Scaling target
target_scaler = StandardScaler()
y_scaled = target_scaler.fit_transform(y.reshape(-1, 1))


# First split: Separate out a test set (5% of the original dataset)
X_temp, X_test, y_temp, y_test = train_test_split(X_scaled_noisy, y_scaled, test_size=(1/20), random_state=16)

# Second split: Split the remaining data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=(1/19), random_state=16)